# Storing skymaps

## Read raw skymap files

In [ ]:
import pickle
from pathlib import Path
import math
import numpy as np


from lsst.sphgeom import Box, ConvexPolygon, UnitVector3d
from math import pi
from lsst.sphgeom import LonLat, UnitVector3d, ConvexPolygon
import yaml
from lsst.sphgeom import UnitVector3d, ConvexPolygon
from skymap_to_poly_coords import load_pickle_skymap

In [ ]:
home_dir = Path.home() 
raw_skymaps_dir = home_dir / "skymap-to-poly-coords" / "tests" / "data" / "raw_skymaps"
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(skymap_path)
lsst_skymap

## Read inner_poly and outer_poly

In [12]:
from skymap_to_poly_coords import load_polygons_ra_dec

skymap_out_dir = "/sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/"
inner_poly_path = Path(skymap_out_dir) / "inner_polys.yaml"
outer_poly_path = Path(skymap_out_dir) / "outer_polys.yaml"

inner_poly_map = load_polygons_ra_dec(inner_poly_path)
outer_poly_map = load_polygons_ra_dec(outer_poly_path)

⚠️ Storing `None` for degenerate tract 0
⚠️ Storing `None` for degenerate tract 18937
✅ Loaded 18938 tract polygons from /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/inner_polys.yaml
✅ Loaded 18938 tract polygons from /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/outer_polys.yaml


In [13]:
# Just patch the polar caps in for now (todo)

from skymap_to_poly_coords.test_utils import get_poly_from_tract_id

inner_poly_map[0] = get_poly_from_tract_id(lsst_skymap, 0, inner=True)
inner_poly_map[len(inner_poly_map)-1] = get_poly_from_tract_id(lsst_skymap, len(inner_poly_map)-1, inner=True)

## Check our saved-and-loaded tracts against the tracts we read via the lsst.skymap package

In [14]:
from skymap_to_poly_coords.test_utils import polys_are_equiv

tracts_to_check = range(0, lsst_skymap._numTracts)

for tract_id in tracts_to_check:
    ground_truth_poly = get_poly_from_tract_id(lsst_skymap, tract_id, inner=True)
    loaded_poly = inner_poly_map[tract_id]
    if not polys_are_equiv(ground_truth_poly, loaded_poly):
        print(f"Tract {tract_id} polygons are NOT equivalent!")
    else:
        continue